In [134]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import preprocessing

import string
import csv

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

from keras.models import Sequential, load_model, Model
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Flatten, MaxPool2D
from keras.optimizers import RMSprop


In [135]:
csv_Train_File_Path = r'C:\Users\Sachin13390\Desktop\Python_Practice_Files\Kaggle_Titanic\all\train - Copy.csv'
csv_Test_File_Path = r'C:\Users\Sachin13390\Desktop\Python_Practice_Files\Kaggle_Titanic\all\test.csv'
# csv_Submission_File_Path = r'C:\Users\Sachin13390\Desktop\Kaggle-Emnist\My_Submission5.csv'

model_File_Path = r'all/Classifier.h5'

In [140]:
def Calculate_Total_Family_Member(df):
    
    Total_Family_Member = []
    
    for i in range (len(df)):
        Total_Family_Member.append(int(df.iloc[i]['SibSp']) + int(df.iloc[i]['Parch']))
        
    df['Total_Family_Member'] = Total_Family_Member
    
    return df


def Get_Average_Age_Per_Class(df, class_type):
    count = 0
    addition = 0.0
    
    for i in range (len(df)):
        
        if  (df.iloc[i]['Pclass'] == class_type ) and (str(type(df.iloc[i]['Age'])) != 'nan') and (float(df.iloc[i]['Age']) > 0.0):        
            addition = addition + float(df.iloc[i]['Age'])
            count = count + 1
            
    average = addition/count
    
    return average


def Missing_Age_Value(df_train, df_test):
    
    unique_class = df_train['Pclass'].unique()
    
    merged_df = frames = [df_train, df_test]
    merged_df = pd.concat(merged_df)
    
    for class_type in unique_class:
        
        average = Get_Average_Age_Per_Class(merged_df, class_type)
            
        for i in range (len(df_train)):
            if (str(df_train.iloc[i]['Pclass']) == str(class_type) ) and (str(df_train.iloc[i]['Age']) == 'nan') :
                df_train.at[i,'Age'] = average
    
    return df_train


def Remove_Unwanted_Column(df, columns_to_remove = []):
    
    if (df['Cabin'].isnull().any().any()) == True:
        df['Cabin'] = df['Cabin'].fillna('Missing')
            
    cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Missing']
    
    for i in range(len(df)):
        for cabin in cabin_list:
            if cabin in df.iloc[i]['Cabin']:
                df.at[i, 'Cabin'] = cabin

    encoder = LabelEncoder()
    df['Cabin']  = encoder.fit_transform(df['Cabin'] )
    
    if (len(columns_to_remove)==0):
        columns_to_remove = ['PassengerId', 'Name', 'Ticket', 'Fare' ]
    
    print(columns_to_remove)
        
    df = df.drop(columns_to_remove, axis=1)
    
    return df
   

    
def Label_Encode_Column(df):
    
    encoder = LabelEncoder()
    
    for i in range (df.shape[1]):
        
        if df.dtypes[i] == 'object':
            
            if (df[df.columns[i]].isnull().any().any()) == True:
                df[df.columns[i]] = df[df.columns[i]].fillna('Missing')
                df[df.columns[i]] = df[df.columns[i]].astype('category')
            
            df[df.columns[i]]  = encoder.fit_transform(df[df.columns[i]] )
    
    return df

def Category_Hot_Encode(df, columns_To_Hot_Encode=[]):
    
    if (len(columns_To_Hot_Encode)==0):
        Pclass = 1
        Sex = 2
        Embarked = 6
        
        columns_To_Hot_Encode = [Embarked, Sex, Pclass]
        
#     one_Hot_Encoder = OneHotEncoder(categorical_features=[1])
#     df = one_Hot_Encoder.fit_transform(df).toarray()
        
    for i in range (len(columns_To_Hot_Encode)):
        print(columns_To_Hot_Encode[i])
        one_Hot_Encoder = OneHotEncoder(categorical_features=[columns_To_Hot_Encode[i]])
        df = one_Hot_Encoder.fit_transform(df).toarray()
    
    df = pd.DataFrame(df)
    
    return df 



def Encoding_Columns(df, columns_to_Encode = []):
    
    if (len(columns_to_Encode)==0):
        columns_to_Encode = ['Pclass']
        
    df = pd.get_dummies(df)
    
    for column in columns_to_Encode:
        encoded_Column = pd.get_dummies(df[column], prefix = column)
        df[encoded_Column.columns] = encoded_Column
        df = df.drop(column, axis=1)
        
    return df   
    


In [141]:
def Data_Modification(df, df_test):
    
    df = Calculate_Total_Family_Member(df)
    
    df = Missing_Age_Value(df, df_test)
    Max_Age_Value = 80.0
    df['Age'] = df['Age']/Max_Age_Value
    
    df = Remove_Unwanted_Column(df)

    df = Encoding_Columns(df)

    return df

In [142]:
df_test = pd.read_csv(csv_Test_File_Path)
df = pd.read_csv(csv_Train_File_Path)

In [143]:
df = Data_Modification(df, df_test)
df

['PassengerId', 'Name', 'Ticket', 'Fare']


,Survived,Age,SibSp,Parch,Cabin,Total_Family_Member,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,0,0.275000,1,0,7,1,0,1,0,0,1,0,0,1
1,1,0.475000,1,0,2,1,1,0,1,0,0,1,0,0
2,1,0.325000,0,0,7,0,1,0,0,0,1,0,0,1
3,1,0.437500,1,0,2,1,1,0,0,0,1,1,0,0
4,0,0.437500,0,0,7,0,0,1,0,0,1,0,0,1
5,0,0.310205,0,0,7,0,0,1,0,1,0,0,0,1
6,0,0.675000,0,0,4,0,0,1,0,0,1,1,0,0
7,0,0.025000,3,1,7,4,0,1,0,0,1,0,0,1
8,1,0.337500,0,2,7,2,1,0,0,0,1,0,0,1
9,1,0.175000,1,0,7,1,1,0,1,0,0,0,1,0


In [144]:
X = df.drop(df.columns[0],axis = 1)
y = df[df.columns[0]]

labelencoder_y_1 = LabelEncoder()
y = labelencoder_y_1.fit_transform(y)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [145]:
classifier = Sequential()

classifier.add(Dense(units = X_train.shape[1], kernel_initializer = 'uniform', activation = 'relu',input_dim = X_train.shape[1]))
classifier.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu' ))
classifier.add(Dense(units = X_train.shape[1], kernel_initializer = 'uniform', activation = 'relu' ))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid' ))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [146]:
classifier.fit(X_train, Y_train,
               batch_size=16, epochs=100,
               verbose=2,
               validation_data=(X_test, Y_test))

Train on 757 samples, validate on 134 samples
Epoch 1/100
 - 1s - loss: 0.6784 - acc: 0.6222 - val_loss: 0.6530 - val_acc: 0.5821
Epoch 2/100
 - 0s - loss: 0.6009 - acc: 0.6539 - val_loss: 0.5654 - val_acc: 0.7388
Epoch 3/100
 - 0s - loss: 0.5109 - acc: 0.7834 - val_loss: 0.4850 - val_acc: 0.7687
Epoch 4/100
 - 0s - loss: 0.4732 - acc: 0.7992 - val_loss: 0.4948 - val_acc: 0.7687
Epoch 5/100
 - 0s - loss: 0.4557 - acc: 0.8018 - val_loss: 0.4625 - val_acc: 0.7761
Epoch 6/100
 - 0s - loss: 0.4483 - acc: 0.8005 - val_loss: 0.4538 - val_acc: 0.7985
Epoch 7/100
 - 0s - loss: 0.4364 - acc: 0.8124 - val_loss: 0.4524 - val_acc: 0.7836
Epoch 8/100
 - 0s - loss: 0.4312 - acc: 0.8137 - val_loss: 0.4552 - val_acc: 0.7985
Epoch 9/100
 - 0s - loss: 0.4309 - acc: 0.8085 - val_loss: 0.4555 - val_acc: 0.7910
Epoch 10/100
 - 0s - loss: 0.4313 - acc: 0.8111 - val_loss: 0.4517 - val_acc: 0.7985
Epoch 11/100
 - 0s - loss: 0.4357 - acc: 0.8058 - val_loss: 0.4508 - val_acc: 0.8060
Epoch 12/100
 - 0s - loss: 0

Epoch 97/100
 - 0s - loss: 0.3843 - acc: 0.8309 - val_loss: 0.4220 - val_acc: 0.8358
Epoch 98/100
 - 0s - loss: 0.3835 - acc: 0.8283 - val_loss: 0.4273 - val_acc: 0.8209
Epoch 99/100
 - 0s - loss: 0.3847 - acc: 0.8336 - val_loss: 0.4278 - val_acc: 0.8284
Epoch 100/100
 - 0s - loss: 0.3871 - acc: 0.8322 - val_loss: 0.4615 - val_acc: 0.8060


In [154]:
classifier.save(model_File_Path)

In [155]:
df_test = pd.read_csv(csv_Test_File_Path)
df = pd.read_csv(csv_Train_File_Path)

In [156]:
df = Data_Modification(df_test,df)


['PassengerId', 'Name', 'Ticket', 'Fare']


In [157]:
y_pred = classifier.predict(df)
y_pred = (y_pred > 0.5)

y_pred

print((y_pred[8][0]))
print((y_pred).shape)

if ((y_pred[8][0]) == True):
    print('aaaaaaaaa')

True
(418, 1)
aaaaaaaaa


In [158]:
csv_Submission_File_Path = 'all\My_Submission.csv'

In [163]:
columns=['PassengerId', 'Survived']
output = []
start_value = 892

threshold = 0.65

y_pred = classifier.predict(df)
y_pred = (y_pred > threshold)

count_Positive = 0

for i in range(len(y_pred)):
#     print(index)
    
    value = 0
    
    if ((y_pred[i][0]) == True):
        value = 1
        count_Positive = count_Positive + 1
    
    row = [start_value, value]
    output.append(row)
    
    start_value = start_value + 1
    
ouput_df = pd.DataFrame(output, columns=columns)

# ouput_df
count_Positive


127

In [164]:
ouput_df.to_csv(csv_Submission_File_Path, index=False)